### NLP Setup

In [ ]:
import pickle
import sys
import spacy

# Load NLP
#sys.path.append('../')

with open('./nlp/nlp.pickle', 'rb') as f:
    nlp = pickle.load(f)

### Import the Template

In [ ]:
from app.templates.meat_sale.nl_template import nl_template

obs = nl_template['obligations']

print('OBLIGATIONS')
for ob in obs:
    print(f'{ob}: {obs[ob]}\n')

In [ ]:
from app.templates.meat_sale.symboleo.contract_template import get_template

contract_template = get_template()

sym_template = contract_template.to_sym()

sym_template_path = './app/templates/meat_sale/symboleo/symboleo_spec.txt'
with open(sym_template_path, 'w') as f:
    f.write(sym_template)

In [ ]:
from app.src.sentence_summarizer import SentenceSummarizer

summarizer = SentenceSummarizer(nlp)

In [ ]:
key = 'CONFIDENTIALITY_TIMEFRAME'
value = 'until 6 months after termination of the contract'

doc = nlp(value)

summarizer.summarize(value)

### Matching

In [ ]:
from spacy.matcher import Matcher

In [ ]:
from app.src.rules.contract_spec.timeframe.timeframe_matcher import get_tf_matcher

matcher = get_tf_matcher(nlp)

matches = matcher(doc)

# get maximal 

for match_id, start, end in matches:
    print(nlp.vocab.strings[match_id], doc[start:end])

In [ ]:
from app.src.rules.contract_spec.predicate_extractor import PredicateExtractor
from app.src.rules.contract_spec.timeframe.timeframe_extractor_builder import TimeFrameExtractorBuilder 
from app.classes.spec.predicate_function import PredicateFunctionHappens
# Get the timeframe extractor

disclosed_event = contract_template.domain_model.events['disclosed'].to_obj()
template = PredicateFunctionHappens(disclosed_event)

extractor = TimeFrameExtractorBuilder.build(nlp, template)


In [ ]:
from app.src.contract_updater import ContractUpdateRequest

req = ContractUpdateRequest(contract_template, key, value, doc)

result = extractor.extract(req)

In [ ]:
print(result.to_sym())

In [ ]:
key = 'DELIVERY_LOCATION'
for x in req.contract.template_strings['obligations']:
    sent = req.contract.template_strings['obligations'][x]
    if key in sent:
        print('found', x)